In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:

from nltk.corpus import wordnet
from nltk.corpus import stopwords

lemmatizer = nltk.stem.WordNetLemmatizer()  # Initiate nltk lemmatizer

In [ ]:
def simple_tokenize(sentence):
    """ Simple function for tokenizing text with nltk """
    return nltk.word_tokenize(sentence, preserve_line=True)

In [ ]:
import math
import os
import pprint
import logging

import datasets
import nltk
import numpy as np
import torch
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import AdamW, get_scheduler, set_seed

from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version

# from args import parse_args
# from data_loader import raw_data_loader, data_processor
from model_loader import model_loader
from rouge_s import py_rouge_scores
from utils import label_smoothed_nll_loss, postprocess_text

import json

from collections import Counter

from nltk.util import ngrams
from nltk import word_tokenize,sent_tokenize

import random
import utils

import datasets
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [ ]:
from transformers import (
    MODEL_MAPPING,
    SchedulerType,
)

# You should update this to your particular problem to have better documentation of `model_type`
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
import argparse
arg_parser = argparse.ArgumentParser(description="BART")
arg_parser.add_argument("--len_input", dest="len_input", type=str, default=None, help="set up prefix input",choices=('no', 'topic', 'length', 'topic-length', 'length-topic', 'simple', 'simple-topic-tagger', 'simple-tagger'))
arg_parser.add_argument("--len_output", dest="len_output", default=None, help="Use the ctrlen model or not", choices=('no', 'topic', 'length', 'topic-length', 'length-topic'))
arg_parser.add_argument("--output_dir", dest="output_dir", type=str, default="./output/1", help="default")
arg_parser.add_argument("--train_file", dest="train_file", type=str, default=None, help="A csv or a json file containing the training data.")
arg_parser.add_argument("--validation_file", dest="validation_file", type=str, default=None, help="A csv or a json file containing the validation data.")
arg_parser.add_argument("--test_file", dest="test_file", type=str, default=None, help="A csv or a json file containing the test data.")
arg_parser.add_argument("--ignore_pad_token_for_loss", dest="ignore_pad_token_for_loss", type=bool, default=True, help="Whether to ignore the tokens corresponding to " "padded labels in the loss computation or not.",)
arg_parser.add_argument("--text_column", dest="text_column", type=str, default="dialogue", help="The name of the column in the datasets containing the full texts (for summarization).")
arg_parser.add_argument("--summary_column", dest="summary_column", type=str, default="summary", help="The name of the column in the datasets containing the summaries (for summarization).")
arg_parser.add_argument("--model_name_or_path", dest="model_name_or_path", type=str, default="facebook/bart-large", help="Path to pretrained model or model identifier from huggingface.co/models.")
arg_parser.add_argument("--model_type", dest="model_type", type=str, default="bart", help="Model type to use if training from scratch.", choices=MODEL_TYPES)
arg_parser.add_argument("--max_source_length", dest="max_source_length", type=int, default=1024, help="default")
arg_parser.add_argument("--source_prefix", dest="source_prefix", type=str, default=None, help="A prefix to add before every source text " "(useful for T5 models).")
arg_parser.add_argument("--preprocessing_num_workers", type=int, default=None, help="The number of processes to use for the preprocessing.")
# arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=lambda x:bool(strtobool(x)), default=True, help="default")
arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=bool, default=None, help="Overwrite the cached training and evaluation sets")
arg_parser.add_argument("--min_target_length", dest="min_target_length", type=int, default=1, help="The minimal total sequence length for target text")
arg_parser.add_argument("--max_target_length", dest="max_target_length", type=int, default=128, help="The maximum total sequence length for target text after "
        "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
        "during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--num_beams", dest="num_beams", type=int, default=4, help="Number of beams to use for evaluation. This argument will be "
        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--learning_rate", dest="learning_rate", type=float, default=5e-5, help="Initial learning rate (after the potential warmup period) to use.")
arg_parser.add_argument("--pad_to_max_length", action="store_true", help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",)
arg_parser.add_argument("--weight_decay", dest="weight_decay", type=float, default=1e-3, help="Weight decay to use.")
arg_parser.add_argument("--label_smoothing", dest="label_smoothing", type=float, default=0.1, help="hyperparameter for label smoothing.")
arg_parser.add_argument("--length_penalty", dest="length_penalty", type=float, default=1.0, help="large - longer sequence, small - shorter sequence")
arg_parser.add_argument("--num_train_epochs", dest="num_train_epochs", type=int, default=15, help="Total number of training epochs to perform.")
arg_parser.add_argument("--per_device_train_batch_size", dest="per_device_train_batch_size", type=int, default=8, help="Batch size (per device) for the training dataloader.")
arg_parser.add_argument("--gradient_accumulation_steps", dest="gradient_accumulation_steps", type=int, default=64, help="Number of updates steps to accumulate before performing a backward/update pass.")
arg_parser.add_argument("--per_device_eval_batch_size", dest="per_device_eval_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--per_device_test_batch_size", dest="per_device_test_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--num_warmup_steps", dest="num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler.")
arg_parser.add_argument("--cache_dir", dest="cache_dir", type=str, default="./output/cache", help="default")
arg_parser.add_argument("--seed", dest="seed", type=int, default=12345, help="default")
# arg_parser.add_argument("-f", required=False) #important
arg_parser.add_argument("--config_name", type=str, default=None, help="Pretrained config name or path if not the same as model_name")
arg_parser.add_argument("--tokenizer_name", type=str, default=None, help="Pretrained tokenizer name or path if not the same as model_name")
arg_parser.add_argument("--use_slow_tokenizer", dest="use_slow_tokenizer", action="store_true", help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).")
arg_parser.add_argument("--max_train_steps", type=int, default=None, help="Total number of training steps to perform. If provided, overrides num_train_epochs.")
arg_parser.add_argument("--lr_scheduler_type", type=SchedulerType, default="linear", help="The scheduler type to use.", choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"])
arg_parser.add_argument("--ctrlen_model", action='store_true', default=False, help="Use the ctrlen model or not")
arg_parser.add_argument("--sim_window_size", type=int, default=5, help="window size for computing loss.")
arg_parser.add_argument("--sim_loss", type=float, default=0, help="the loss weight for similarity scores.")
arg_parser.add_argument("--special_len_token_init", type=str, default=None, help="ways to initialize special token for length (random, zero, token_embs)")
arg_parser.add_argument("--embedding_lr", type=float, default=5e-5, help="Initial learning rate for embedding layers.")
arg_parser.add_argument("--len_start", type=int, default=1, help="start length.")
arg_parser.add_argument("--len_end", type=int, default=100, help="end length.")
arg_parser.add_argument("--data_aug",action='store_true',default=False,help="whether to perform data augmentation or not")
arg_parser.add_argument("--pred_len", action='store_true', default=False, help="whether to use the golden length or predicted length")
arg_parser.add_argument("--shuffle", action='store_true', default=False, help="whether to shuffle the dataset to balance train/validation/test")
arg_parser.add_argument("--debug", action='store_true', default=False, help="Use the debug mode or not")

arg_parser.add_argument("--topic_tagger", dest="topic_tagger", type=bool, default=None, help="Use topic tag [TAG] or not")

In [ ]:
args = arg_parser.parse_args('')

In [ ]:
args.train_file = "./data/dialogtest/dialogsum.train.jsonl"
args.validation_file = "./data/dialogtest/dialogsum.dev.jsonl"
args.test_file = "./data/dialogtest/dialogsum.test.jsonl"
args.text_column = "dialogue"
args.summary_column = "summary"
args.model_name_or_path = "facebook/bart-large"
args.model_type = "bart"
args.max_source_length = 1024
args.min_target_length = 1
args.max_target_length = 128
args.num_beams = 4
args.learning_rate = 5e-5
args.weight_decay = 1e-3
args.label_smoothing = 0.1
args.length_penalty = 1.0 
args.num_train_epochs = 3
args.per_device_train_batch_size = 2 
args.gradient_accumulation_steps = 64 
args.per_device_eval_batch_size = 8 
args.per_device_test_batch_size = 8 
args.num_warmup_steps = 0 
args.cache_dir = "./output/cache"
args.overwrite_cache = True
args.seed = 12345

args.len_input = 'topic-length'
args.len_output = 'no'
args.output_dir = "./output/1-bart-baseline-loss"

args.topic_tagger = True

In [ ]:
print(args.len_input)
print(args.topic_tagger)

In [ ]:
def simple_tokenize(sentence):
    """ Simple function for tokenizing text with nltk """
    return nltk.word_tokenize(sentence)

def nltk_to_pos(pos):
    """ Simple function for converting nltk pos to wordnet pos"""
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_text(text):
    """ Function to lemmatize text according to the wordnet POS of each token """

    tokenized_text = nltk.word_tokenize(text)
    POS_assigned_text = nltk.pos_tag(tokenized_text)

    available_POS = map(lambda x: (x[0], nltk_to_pos(x[1])), POS_assigned_text)

    lemmatized_text = [token if pos is None
                       else lemmatizer.lemmatize(token, pos)
                       for token, pos in available_POS]

    return lemmatized_text

def build_tagger(original_tokens,lemmatized_tokens, topic_list, idx):
    tagged_tokens = []
    # Extract all the seed words according to the corresponding topic
    token_topics = topic_list
    original_list = original_tokens[idx]

    for j, token in enumerate(lemmatized_tokens[idx]):
        # If the lemmatized form of the token is in topic seeds, tag the original token
        if token.lower() in token_topics:
            # print(token.lower())
            if token.lower() not in stopwords.words('english'):
                # print("="*100)
                # print(token.lower())
                original_list[j] = '[TAG]' + original_list[j] + '[TAG]'

    tagged_tokens.append(" ".join(original_list))
    return tagged_tokens

In [ ]:
def load_from_dialogsum(args, file_path):
    ''' load dialoguesum jsonl data '''

    data = []

    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))

    id_list = [sample['fname'] for sample in data]
    dialogue_list = [sample['dialogue'] for sample in data]

    if 'summary' in data[0]:
        # summary
        summary_list = [sample['summary'] for sample in data]
        # topic
        topic_list = [sample['topic'] for sample in data]

    elif 'summary1' in data[0]:

        id_list1 = [id+"_sum1" for id in id_list]
        id_list2 = [id+"_sum2" for id in id_list]
        id_list3 = [id+"_sum3" for id in id_list]

        id_list = id_list1 + id_list2 + id_list3
        dialogue_list = dialogue_list + dialogue_list + dialogue_list

        # summary
        summary_list1 = [sample['summary1'] for sample in data]
        summary_list2 = [sample['summary2'] for sample in data]
        summary_list3 = [sample['summary3'] for sample in data]

        summary_list = summary_list1 + summary_list2 + summary_list3

        # topic
        topic_list1 = [sample['topic1'] for sample in data]
        topic_list2 = [sample['topic2'] for sample in data]
        topic_list3 = [sample['topic3'] for sample in data]

        topic_list = topic_list1 + topic_list2 + topic_list3

    if args.topic_tagger:
        topic_tagger = []
        original_tokens = [simple_tokenize(x) for x in dialogue_list]
        lemmatized_tokens = [lemmatize_text(x) for x in dialogue_list]
        for i in range(len(lemmatized_tokens)):
            tagger = build_tagger(original_tokens, lemmatized_tokens, topic_list[i], i)
            topic_tagger.extend(tagger)

        data_dict = {'id': id_list,
                     'dialogue': topic_tagger,
                     'summary': summary_list,
                     'topic': topic_list}
    else:
        data_dict = {'id': id_list,
                     'dialogue': dialogue_list,
                     'summary': summary_list,
                     'topic': topic_list}

    data_dict = Dataset.from_dict(data_dict)

    return data_dict

In [ ]:
def raw_data_loader(args):
    ''' load raw datasets from csv files '''

    data_files = {}
    if args.train_file is not None:
        data_files["train"] = args.train_file
    if args.validation_file is not None:
        data_files["validation"] = args.validation_file
    if args.test_file is not None:
        data_files["test"] = args.test_file

    if 'dialogsum' in args.train_file:
        train_dict = load_from_dialogsum(args, args.train_file)
        val_dict   = load_from_dialogsum(args, args.validation_file)
        test_dict  = load_from_dialogsum(args, args.test_file)

    train_dict = utils.len_adjust(args, train_dict, 'train')
    val_dict   = utils.len_adjust(args, val_dict, 'val')
    test_dict  = utils.len_adjust(args, test_dict, 'test')

    raw_datasets = datasets.DatasetDict({"train":train_dict, "validation":val_dict, "test":test_dict})

    return raw_datasets

In [ ]:
dialogsum = raw_data_loader(args)
dialogsum

In [ ]:
def data_processor(logger, args, accelerator, raw_datasets, tokenizer, model):
    ''' prepare dataset format for train/val/test '''
    def preprocess_function(examples):

        # summary - target
        targets = examples[summary_column]
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        if args.ctrlen_model:
            gold_sum_len = [len(item) for item in labels['attention_mask']]

        # dialogue - input
        inputs = examples[text_column]
        new_inputs = []
        for i, inp in enumerate(inputs):
            if args.ctrlen_model:
                if 'pred_len' in examples:
                    new_inputs.append(prefix + "<len_{}> ".format(examples['pred_len'][i]) + inp)

                else:
                    new_inputs.append(prefix + "<len_{}> ".format(gold_sum_len[i]) + inp)
            else:
                new_inputs.append(prefix + inp)

        inputs = new_inputs
        model_inputs = tokenizer(inputs, max_length=args.max_source_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]
        
        model_inputs["labels"] = labels["input_ids"]

        if args.ctrlen_model:
            model_inputs["gold_len"] = gold_sum_len

        return model_inputs

    prefix = args.source_prefix if args.source_prefix is not None else ""

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    column_names = raw_datasets["train"].column_names

    # Get the column names for input/target.
    text_column = args.text_column
    if text_column not in column_names:
        raise ValueError(
            f"--text_column' value '{args.text_column}' needs to be one of: {', '.join(column_names)}"
        )

    summary_column = args.summary_column
    if summary_column not in column_names:
        raise ValueError(
            f"--summary_column' value '{args.summary_column}' needs to be one of: {', '.join(column_names)}"
        )

    # Temporarily set max_target_length for training.
    max_target_length = args.max_target_length
    padding = "max_length" if args.pad_to_max_length else False

    

    with accelerator.main_process_first():
        processed_datasets = raw_datasets.map(
            preprocess_function,
            batched=True,
            batch_size=1000,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )

    train_dataset = processed_datasets["train"]
    eval_dataset  = processed_datasets["validation"]
    test_dataset  = processed_datasets["test"]

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 1):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    label_pad_token_id = -100 if args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if accelerator.use_fp16 else None,
    )

    train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size)
    eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)
    test_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=args.per_device_test_batch_size)

    return (train_dataloader, eval_dataloader, test_dataloader), (train_dataset, eval_dataset, test_dataset)

In [ ]:
raw_datasets = raw_data_loader(args)

In [ ]:
accelerator = Accelerator(mixed_precision="fp16")
logger.info(accelerator.state)

In [ ]:
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
accelerator.is_local_main_process
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()
set_seed(args.seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
accelerator.is_main_process
os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [ ]:
config, tokenizer, model = model_loader(accelerator, logger, args)

In [ ]:
dataloader, processed_dataset = data_processor(logger, args, accelerator, raw_datasets, tokenizer, model)
train_dataloader, eval_dataloader, test_dataloader = dataloader
train_dataset, _, _ = processed_dataset

In [ ]:
# = = = Training Preparation = = =
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]

if args.ctrlen_model: 
    no_decay_emb_matrix = ["bias", "LayerNorm.weight", "shared"]
else:
    no_decay_emb_matrix = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay_emb_matrix)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

if args.ctrlen_model:
    if args.model_type == 'bart': 
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.model.shared.parameters(),
            "lr": args.embedding_lr}])
    elif args.model_type == 't5':
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.shared.parameters(),
            "lr": args.embedding_lr}])
    else:
        raise ValueError('{} model type not implemented'.format(args.model_type))

# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

In [ ]:
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

In [ ]:
# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ", disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses  = []
best_r2_f1  = None
best_epoch  = 0

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

In [ ]:
# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * \
    accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(
    f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(
    f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(
    f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ",
                    disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses = []
best_r2_f1 = None
best_epoch = 0

# edit #
if args.model_type == 'bart' or args.model_type == 't5':
    # task_specific_params = model.module.config.task_specific_params
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    # model.module.config.update(params)
    model.config.update(params)
else:
    raise ValueError(
        '{} model type not implemented'.format(args.model_type))

loss_list = []
train_loss_list = []
val_loss_list = []

# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Train =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
for epoch in range(args.num_train_epochs):
    # train
    model.train()
    epoch_loss = 0.0
    for step, batch in enumerate(train_dataloader):

        if args.ctrlen_model:  # CTRLen model
            outputs, loss = model(batch, tokenizer)
        # w/ and w/o label smoothing (always better with label smoothing)
        else:
            if args.label_smoothing == 0:
                outputs = model(**batch)
                loss = outputs.loss
            else:
                outputs = model(**batch)
                output_logits = outputs.logits
                hidden_states = outputs.encoder_last_hidden_state
                print(f"logits: {output_logits.shape}")
                print("="*100)
                # print(f"hidden states: {hidden_states.shape}")
                print(f"loss: {outputs.loss}")
                print(outputs.keys())
                print("="*100)
                output_probs = torch.nn.functional.log_softmax(
                    output_logits, dim=-1)
                # edit #
                # output_probs = output_probs.view(-1,
                #                                  model.module.config.vocab_size)
                output_probs = output_probs.view(-1,
                                                 model.config.vocab_size)

                gt_logits = batch['labels']
                print(f"label: {gt_logits.shape}")
                print("="*100)
                gt_logits = gt_logits.view(-1)

                print(f"output_probs: {output_probs.shape}")
                print("-"*100)
                print(f"gt_logits: {gt_logits.shape}")
                print("="*100)
                
                loss, nll = label_smoothed_nll_loss(
                    output_probs, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)

                print(f"loss_fn: {loss}")
                print("-"*100)
                print(f"nll: {nll}")
                print("="*100)

        acc_losses.append(loss.item())
        loss_list.append(loss)
        epoch_loss += loss.item()
        loss = loss / args.gradient_accumulation_steps
        print(f"loss_grad: {loss}")
        accelerator.backward(loss)
        break

        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            progress_bar.set_postfix(lr=lr_scheduler.get_last_lr()[
                                     0], loss=np.mean(acc_losses[-50:]))
            completed_steps += 1
            train_loss_list.append(epoch_loss/len(batch))

        if completed_steps >= args.max_train_steps:
            break

In [ ]:
# hidden_states[0].shape

In [ ]:
# output_logits.shape

In [ ]:
gt_logits

In [ ]:
idx = gt_logits.ne(-100).sum(-1)

In [ ]:
idx.item()

In [ ]:
output_logits.shape

In [ ]:
output_logits[0].shape

In [ ]:
output_logits[0]

In [ ]:
output_logits[1]

In [ ]:
output_probs.shape

In [ ]:
x1 = output_probs[:output_logits.shape[1]]
x2 = output_probs[output_logits.shape[1]:]

In [ ]:
x1.shape

In [ ]:
x2.shape

In [ ]:
X = torch.nn.functional.log_softmax(output_logits, dim=1)

In [ ]:
X_probs = X.view(-1, model.config.vocab_size)

In [ ]:
X_probs.shape

In [ ]:
Y = torch.exp(X_probs[0])

In [ ]:
Y

In [ ]:
_, predicted_indices = torch.max(X_probs, dim=1)

In [ ]:
predicted_indices.shape

In [ ]:
predicted_indices

In [ ]:
Y_decoded = tokenizer.batch_decode(
            predicted_indices[:output_logits.shape[1]], skip_special_tokens=True)

In [ ]:
Y_decoded

In [ ]:
gt = gt_logits.cpu().numpy()

In [ ]:
gt

In [ ]:
labels = np.where(gt != -100, gt,
                              tokenizer.pad_token_id)

In [ ]:
labels

In [ ]:
label_decoded = tokenizer.batch_decode(
            labels[:output_logits.shape[1]], skip_special_tokens=True)

In [ ]:
label_decoded

In [ ]:
decoded_preds, decoded_labels = postprocess_text(
            Y_decoded, label_decoded)

In [ ]:
len(decoded_preds)

In [ ]:
len(decoded_labels)

In [ ]:
decoded_labels

In [ ]:
decoded_labels

In [ ]:
decoded_clean = [item for item in decoded_labels if item != '']

In [ ]:
len(decoded_clean)

In [ ]:
decoded_preds[:len(decoded_clean)]

In [ ]:
gt_logits[0]

In [ ]:
gt_logits.ndim

In [ ]:
def label_smoothed_nll_loss(lprobs, target, epsilon, ignore_index=-100):
    '''
        loss with label smoothing
        from fairseq, edit by Bin
    '''

    lprobs = lprobs[~target.eq(-100)]
    target = target[~target.eq(-100)]

    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(-1)

    nll_loss = -lprobs.gather(dim=-1, index=target)
    smooth_loss = -lprobs.sum(dim=-1, keepdim=True)

    # mean()? Scared to break other math.
    # bin: change from sum to mean
    nll_loss = nll_loss.mean()
    smooth_loss = smooth_loss.mean()
    eps_i = epsilon / lprobs.size(-1)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss, nll_loss

In [ ]:
lprobs = output_probs[~gt_logits.eq(-100)]
lprobs

In [ ]:
target = gt_logits[~gt_logits.eq(-100)]
target

In [ ]:
lprobs.shape, target.shape

In [ ]:
lprobs.ndim, target.ndim

In [ ]:
target.dim() == lprobs.dim() - 1

In [ ]:
if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(-1)

In [ ]:
lprobs.shape, target.shape

In [ ]:
lprobs[5][10431]

In [ ]:
target[:10]

In [ ]:
for i in range(0, 10):
   print(nll_loss[i].item() == -lprobs[i][target[i]].item())

In [ ]:
nll_loss = -lprobs.gather(dim=-1, index=target)
nll_loss.shape, nll_loss[:5]

In [ ]:
smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
smooth_loss.shape, smooth_loss[:5]

In [ ]:
nll_loss = nll_loss.mean()
smooth_loss = smooth_loss.mean()
nll_loss, smooth_loss

In [ ]:
epsilon = args.label_smoothing

In [ ]:
eps_i = epsilon / lprobs.size(-1)
loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
eps_i, loss

In [ ]:
output_probs.shape

In [ ]:
gt_logits.shape

In [ ]:
for step, batch in enumerate(eval_dataloader):
    print(step, len(batch), len(batch['input_ids']), len(batch['attention_mask']), len(batch['labels']), len(batch['decoder_input_ids']))
    if step == 1:
        break

In [ ]:
# # =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = EVAL =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
model.eval()
val_predict = []
val_groundtruth = []
for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        val_loss = []
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )

        # print(generated_tokens)
        # print("="*100)
        
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        
        # print(generated_tokens)
        # print("="*100)
        
        labels = batch["labels"]
        if not args.pad_to_max_length:
            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)
        
        print(generated_tokens)
        print("="*100)
        print(labels)
        print("="*100)
        # loss, _ = label_smoothed_nll_loss_gen(generated_tokens, labels, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
        # break    
        # loss, _ = map(label_smoothed_nll_loss, generated_tokens, labels, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
        # val_loss.extend(loss)
        
        generated_tokens = accelerator.gather(
            generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()

        if args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels,
                              tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        # print(generated_tokens[0])
        # print("="*100)
        # print(labels[0])
        # loss, _ = label_smoothed_nll_loss(generated_tokens[0], labels[0], args.label_smoothing, ignore_index=tokenizer.pad_token_id)
        # break 
        
        decoded_preds = tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(
            labels, skip_special_tokens=True)
        
        print(decoded_preds)
        print("="*100)
        print(decoded_labels)
        print("="*100)
        
        decoded_preds, decoded_labels = postprocess_text(
            decoded_preds, decoded_labels)

        print(decoded_preds)
        print("="*100)
        print(decoded_labels)
        print("="*100)
        if step == 1:
            break

        val_predict.extend(decoded_preds)
        val_groundtruth.extend(decoded_labels)

if args.len_output == 'real':
    new_val_predict = []
    for sample in val_predict:
        try:
            gen_sum = sample.split('Summary: ')[2]
            new_val_predict.append(gen_sum)
        except:
            new_val_predict.append(sample)
    val_predict = new_val_predict
else:
    new_val_predict = val_predict

logger.info("")
logger.info("Rouge score on val set after epoch {}".format(epoch+1))
eval_results = py_rouge_scores(val_predict, val_groundtruth)

if best_r2_f1 is None:
    best_r2_f1 = eval_results
if eval_results['rouge-2']['f'] >= best_r2_f1['rouge-2']['f']:
    best_r2_f1 = eval_results
    best_epoch = epoch + 1

    os.makedirs(args.output_dir+'/best', exist_ok=True)
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args.output_dir+'/best', save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args.output_dir+'/best')
    # save vocab
    vocab = tokenizer.vocab.copy()
    vocab = {k: v for k, v in sorted(
        vocab.items(), key=lambda item: item[1])}
    with open(args.output_dir + '/best/vocab.txt', 'w') as f:
        for word, index in vocab.items():
            # it lead to encoding bug on some machines, so i add this line
            word = word.encode('ascii', 'ignore').decode('ascii')
            f.write(str(index) + ': ' + word + '\n')

# = = = = = = = = = = = = = = = = = = = = = = = = =
logger.info(
    "Current Best Validation Result is at epoch {}".format(best_epoch))
py_rouge_scores(None, None, best_r2_f1)

In [ ]:
loss_list[0].item()

In [ ]:
loss_list[0].item() / 64

In [ ]:
acc_losses[0]

In [ ]:
len(loss_list)

In [ ]:
len(acc_losses)

In [ ]:
# for epoch in range(args.num_train_epochs):
#     # train
#     model.train()
#     epoch_loss = 0.0
#     for step, batch in enumerate(train_dataloader):

In [ ]:
step_list = []
for step, batch in enumerate(train_dataloader):
    step_list.append(step)

In [ ]:
step_list[-1]

In [ ]:
train_loss_list[0]

In [ ]:
len(train_loss_list)

In [ ]:
# loss

In [ ]:
# loss_list

In [ ]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Test =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = 
# load best model
logger.info("Loading Best Result is at epoch {} for Testing".format(best_epoch))

unwrapped_model = accelerator.unwrap_model(model)
config          = config.from_pretrained(args.output_dir+'/best')
tokenizer       = tokenizer.from_pretrained(args.output_dir+'/best', config=config)
unwrapped_model = unwrapped_model.from_pretrained(args.output_dir+'/best', config=config)
model           = accelerator.prepare(unwrapped_model)

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

# start Test 
logger.info("Collecting Testing Result...")
model.eval()

test_predict     = []
test_groundtruth = []
for step, batch in enumerate(tqdm(test_dataloader, leave=False)):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = batch["labels"]

        if not args.pad_to_max_length:
            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()

        if args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds  = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        decoded_preds  = [' '.join(sent.split('\n')) for sent in decoded_preds]
        decoded_labels = [' '.join(sent.split('\n')) for sent in decoded_labels]

        test_predict.extend(decoded_preds)
        test_groundtruth.extend(decoded_labels)

print(raw_datasets['test']['dialogue'][0])

if args.len_output == 'real':
    new_test_predict = []
    for sample in test_predict:
        try:
            gen_sum = sample.split('Summary: ')[2]
            new_test_predict.append(gen_sum)
        except:
            new_test_predict.append(sample)
    test_predict = new_test_predict

logger.info("")
logger.info("ROUGE score on test set")
test_scores = py_rouge_scores(test_predict, test_groundtruth)
logger.info("")


# Save generated summaries
if args.len_input == 'predict':
    os.makedirs(args.output_dir+'/predict_gen_samples', exist_ok=True)
else:
    os.makedirs(args.output_dir+'/gen_samples', exist_ok=True)

for i in range(len(test_predict)):
    test_id        = raw_datasets['test']['id'][i]
    test_dialogue  = raw_datasets['test']['dialogue'][i]
    test_summary   = raw_datasets['test']['summary'][i]
    test_predict_s = test_predict[i]

    if args.len_input == 'predict':
        with open(args.output_dir+'/predict_gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)
    else:
        with open(args.output_dir+'/gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)